<a href="https://colab.research.google.com/github/aniloracz/SD-LoRA-Trainer/blob/main/PEACH_Cuaderno_Base_Taller_de_LoRA_para_Stable_Diffusion_CINTIA_2024_PEACH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **🧮 Taller de LoRA para Stable Diffusion - CINTIA 2024**

---

*Realizado por **Carolina Zunino** y **Diego Miranda** - (carolinazuninoruso@gmail.com , diego.miranda@mungo.tv) Universidad ORT, Facultad de Diseño*

 *Basado en el trabajo de [HollowStrawberry](https://github.com/hollowstrawberry/kohya-colab), [Kohya_ss](https://github.com/kohya-ss/sd-scripts), [Linaqruf](https://colab.research.google.com/github/Linaqruf/kohya-trainer/blob/main/kohya-LoRA-dreambooth.ipynb#scrollTo=-Z4w3lfFKLjr), [uYouUs](https://github.com/uYouUs)*

---

### **En este taller vamos a trabajar sobre el entrenamiento de un LoRA para el modelo de Stable Diffusion 1.5 con la cara de Mr Bean, utilizando Google Colab, Google Drive y Hugging Face**

Este cuaderno es una guía para preparar un dataset de fotos etiquetadas, luego utlizarlas en el entrenamiento de un LoRA (low rank adaptation) y finalmente generar una imágen sintética.

Stable Diffusion, SD, es un modelo de generación de imágenes que a partir de una imágen inicial compuesta por pixeles random, y una descripción en texto, va extrayendo "ruido" de la imágen en sucesivos pasos hasta llegar a una versión final que coincide con la descripción (prompt)

Un LoRA es un modelo de Stable Diffusion minúsculo y muy específico que permite modificar de manera muy controlada las imágenes creadas. Puede ser usado de muchas formas, para transferencia de estilos artísticos, de objetos específicos o de rasgos faciales.

---

*Nota: SD, LoRAS y los modelos de aprendizaje automático son tecnologías muy nuevas que están en constante cambio y actualización, por eso es posible que este cuaderno deje de funcionar en cuanto alguna de las tecnologías utilizadas realice una actualización de sus librerías. En un principio debería poderse realizar todo el entrenamiento sin tener que comprar créditos de Google Colab, pero esto puede cambiar en cualquier momento también. Los autores y colaboradores no se hacen responsables del funcionamiento ni de las fotos sintéticas realizadas con este cuaderno.*

#🧮 1 - Armado del Proyecto y Dataset






##🧮 1.1 - Nomenclatura del proyecto

Para trabajar en este proyecto vamos a utilizar un nombre único que servirá para conectar los archivos de nuestro GDrive y luego activar nuestro LoRA.

Por ejemplo si vamos a usar las fotos provistas en el ejemplo de Mr Bean, el nombre será  "**mrbeanface**", ese nombre deberá ser el mismo para la carpeta de Gdrive y para la confuguración de proyectos. Podemos elegir cualquier nombre, pero deberá ser siempre el mismo para todo el cuaderno.

Una vez elegido el nombre creamos una estructura de carpetas en Google Drive donde se alojará nuestro proyecto y deberá ser así:

/My Drive/Loras/mrbeanface/dataset

Una vez creadas estas carpetas avanzamos al siguiente paso.




##🧮 1.2 - Preparación de imágenes

Para trabajar en este cuaderno precisaremos 15 fotos de la persona que vamos a usar en nuestro LoRA, tienen que ser con distinto ángulo, distinta luz, en distintos lugares, preferentemente algunas de cuerpo entero o plano medio, no todas selfies, y es muy importante que NO HAYA OTRAS CARAS en la foto. Para mejor resultado tenemos que elegir fotos de la misma época y look (barba, arrugas, lentes, corte y color de pelo)

Las extensiones permitidas son ('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.webp', '.avif') y todos los nombres de foto deben ser distintos (mismo nombre y distinta extensión no permitido)

Es importante que las fotos tengan alta calidad.

Una vez que tenemos nuestras 15 imágenes correctamente seleccionadas las subimos a nuestra carpeta dataset en GDrive.

Las imágenees provistas en el ejemplo se encuentran en [FOTOS de EJEMPLO](https://drive.google.com/drive/folders/1xefxxRPf9dnlubmBC3Z3TOqWHXl7SdL1?usp=sharing) . Para trabajar con ellas simplemente las descargamos y las volvemos a subir a nuestra carpeta dataset en nuestro drive.


##🧮 1.3 - Etiquetado de Imágenes - 🔌Conectarse a CPU

Para este paso nos vamos a conectar a un entorno de ejecución con CPU.

Utilizaremos una librería llamada BLIP, que va a escanear nuestras imágenes, reconocer figuras y describirlas, creando en nuestra carpeta dataset un archivo .txt para cada imágen. A cada descripción además le agrega nuestra palabra clave al comienzo como activador. Por ejemplo: mrbeanface, a man with a tie...

Esta dupla de imágen-descripción será nuestro DATASET para entrenar el LoRA.

Corremos la celda de código debajo y verificamos que se crearon los archivos .txt en el drive. Debemos estar atentos a autorizar la conexión con google drive.




In [ ]:
#@markdown Nombre del proyecto:

project_name = "mrbeanface"  # @param {type:"string"}

project_name = project_name.strip()

##@markdown Esta es la estructura de drive que vamos a usar donde project_name es mrbeanface u otro nombre elegido
folder_structure = "Organize by project (MyDrive/Loras/project_name/dataset)"
## @param ["Organize by category (MyDrive/lora_training/datasets/project_name)", "Organize by project (MyDrive/Loras/project_name/dataset)"]



In [ ]:

# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Install Necessary Packages
!pip install transformers==4.31.0
!pip install torch torchvision
!pip install Pillow

# Step 3: Import Libraries
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
import os

# Step 4: Set Up the Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Step 5: Load the Model and Processor
processor = BlipProcessor.from_pretrained('Salesforce/blip-image-captioning-base')
model = BlipForConditionalGeneration.from_pretrained('Salesforce/blip-image-captioning-base')
model.to(device)

# Step 6: Specify the Images Folder Path
# Replace 'path_to_your_images_folder' with your actual folder path
images_folder = '/content/drive/MyDrive/Loras/'+project_name+'/dataset' # CAMBIAR ACA

# Step 7: Generate Captions for Each Image
image_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.webp')
image_files = [f for f in os.listdir(images_folder) if f.lower().endswith(image_extensions)]

print(f"Found {len(image_files)} images in the folder.")

for image_file in image_files:
    image_path = os.path.join(images_folder, image_file)
    try:
        # Open and process the image
        image = Image.open(image_path).convert('RGB')
        inputs = processor(images=image, return_tensors="pt").to(device)

        # Generate the caption
        out = model.generate(**inputs)
        caption = project_name+", " + processor.decode(out[0], skip_special_tokens=True)

        # Save the caption to a .txt file
        txt_file = os.path.splitext(image_path)[0] + '.txt'
        with open(txt_file, 'w') as f:
            f.write(caption)

        print(f"Caption generated and saved for {image_file}")
    except Exception as e:
        print(f"Error processing {image_file}: {e}")

ValueError: mount failed



---
✈ Fin parte 1 - Desconectar y eliminar nuestro entorno de ejecución


---



#  🧮 2 - Configuración y entrenamiento del modelo creando un LoRA con nuestro dataset en un entorno virtual con GPU - 🔌Conectarse a T4


Para correr este entrenador debemos conectarnos a una maquina virtual T4.
Luego elegimos qué modelo de SD vamos a entrenar, hay varios modelos que van actualizándose con el paso del tiempo, los modelos a la vez que se van volviendo más precisos y más definidos, también se van volviendo más pesados y lentos, requiriendo mucha capacidad de procesamiento y mucho tiempo de entrenamiento para lograr excelentes resultados.

Vamos a trabajar con el modelo SD1.5, que es más pequeño y no lleva tanto tiempo de entrenamiento para el LoRA.

Además configuramos una cantidad de parámetros que definen cuántas veces se va a correr el entrenamiento con nuestro dataset, qué intensidad de aprendizaje queremos que tenga, cuánta memoria ram tenemos disponible. Estos parámetros modifican los cálculos que el modelo va a realizar en su interior llevando a distintos ajustes en el peso y sesgo de los cálculos y en la cantidad de repeticiones.

En este caso vamos a usar el entrenador con los valores ya configurados, corremos la celda debajo para que se guarde la configuración.

En el último paso cuando los valores ya están configurados seo descarga el modelo SD 1.5 y muchas otras librerías pesadas y se entrena al modelo usando nuestras fotos, el proceso se corre en épocas y puede demorar hasta 30 minutos.

Al terminar, veremos en nuestra carpeta GDrive una carpeta output con los archivos tensorflow, que son nuestro arvhivos LoRA.

In [ ]:
import os
import re
import toml
from time import time
from IPython.display import Markdown, display

# These carry information from past executions
if "model_url" in globals():
  old_model_url = model_url
else:
  old_model_url = None
if "dependencies_installed" not in globals():
  dependencies_installed = False
if "model_file" not in globals():
  model_file = None

# These may be set by other cells, some are legacy
if "custom_dataset" not in globals():
  custom_dataset = None
if "override_dataset_config_file" not in globals():
  override_dataset_config_file = None
if "override_config_file" not in globals():
  override_config_file = None
if "optimizer" not in globals():
  optimizer = "AdamW8bit"
if "optimizer_args" not in globals():
  optimizer_args = None
if "continue_from_lora" not in globals():
  continue_from_lora = ""
if "weighted_captions" not in globals():
  weighted_captions = False
if "adjust_tags" not in globals():
  adjust_tags = False
if "keep_tokens_weight" not in globals():
  keep_tokens_weight = 1.0

COLAB = True # low ram
XFORMERS = True
SOURCE = "https://github.com/uYouUs/sd-scripts"
COMMIT = None
BETTER_EPOCH_NAMES = True
LOAD_TRUNCATED_IMAGES = True


#@markdown ### ⚡ Setup  - 🔌 T4
#@markdown Your project name will be the same as the folder containing your images. Spaces aren't allowed.
project_name = "mrbeanface" #@param {type:"string"}
project_name = project_name.strip()
#@markdown The folder structure doesn't matter and is purely for comfort. Make sure to always pick the same one. I like organizing by project.
folder_structure = "Organize by project (MyDrive/Loras/project_name/dataset)" #@param ["Organize by category (MyDrive/lora_training/datasets/project_name)", "Organize by project (MyDrive/Loras/project_name/dataset)"]
#@markdown Decide the model that will be downloaded and used for training. These options should produce clean and consistent results. You can also choose your own by pasting its download link.
training_model = "Stable Diffusion (sd-v1-5-pruned-noema-fp16.safetensors)" #@param ["Anime (animefull-final-pruned-fp16.safetensors)", "AnyLora (AnyLoRA_noVae_fp16-pruned.ckpt)", "Stable Diffusion (sd-v1-5-pruned-noema-fp16.safetensors)"]
optional_custom_training_model_url = "" #@param {type:"string"}
custom_model_is_based_on_sd2 = False #@param {type:"boolean"}

if optional_custom_training_model_url:
  model_url = optional_custom_training_model_url
elif "AnyLora" in training_model:
  model_url = "https://huggingface.co/Lykon/AnyLoRA/resolve/main/AnyLoRA_noVae_fp16-pruned.ckpt"
elif "Anime" in training_model:
  model_url = "https://huggingface.co/hollowstrawberry/stable-diffusion-guide/resolve/main/models/animefull-final-pruned-fp16.safetensors"
else:
  model_url = "https://huggingface.co/hollowstrawberry/stable-diffusion-guide/resolve/main/models/sd-v1-5-pruned-noema-fp16.safetensors"

#@markdown ### ⚡ Processing
#@markdown Resolution of 512 is standard for Stable Diffusion 1.5. Higher resolution training is much slower but can lead to better details. <p>
#@markdown Images will be automatically scaled while training to produce the best results, so you don't need to crop or resize anything yourself.
resolution = 512 #@param {type:"slider", min:512, max:1024, step:128}
#@markdown This option will train your images both normally and flipped, for no extra cost, to learn more from them. Turn it on specially if you have less than 20 images. <p>
#@markdown **Turn it off if you care about asymmetrical elements in your Lora**.
flip_aug = True #@param {type:"boolean"}
#markdown Leave empty for no captions.
caption_extension = ".txt" #param {type:"string"}
#@markdown Shuffling anime tags in place improves learning and prompting. An activation tag goes at the start of every text file and will not be shuffled.
shuffle_tags = True #@param {type:"boolean"}
shuffle_caption = shuffle_tags
activation_tags = "1" #@param [0,1,2,3]
keep_tokens = int(activation_tags)

#@markdown ### ⚡ Steps <p>
#@markdown Your images will repeat this number of times during training. I recommend that your images multiplied by their repeats is between 200 and 400.
num_repeats = 20 #@param {type:"number"}
#@markdown Choose how long you want to train for. A good starting point is around 10 epochs or around 2000 steps.<p>
#@markdown One epoch is a number of steps equal to: your number of images multiplied by their repeats, divided by batch size. <p>
preferred_unit = "Epochs" #@param ["Epochs", "Steps"]
how_many = 10 #@param {type:"number"}
max_train_epochs = how_many if preferred_unit == "Epochs" else None
max_train_steps = how_many if preferred_unit == "Steps" else None
#@markdown Saving more epochs will let you compare your Lora's progress better.
save_every_n_epochs = 1 #@param {type:"number"}
keep_only_last_n_epochs = 10 #@param {type:"number"}
if not save_every_n_epochs:
  save_every_n_epochs = max_train_epochs
if not keep_only_last_n_epochs:
  keep_only_last_n_epochs = max_train_epochs
#@markdown Increasing the batch size makes training faster, but may make learning worse. Recommended 2 or 3.
train_batch_size = 2 #@param {type:"slider", min:1, max:8, step:1}

#@markdown ### ⚡ Learning
#@markdown The learning rate is the most important for your results. If you want to train slower with lots of images, or if your dim and alpha are high, move the unet to 2e-4 or lower. <p>
#@markdown The text encoder helps your Lora learn concepts slightly better. It is recommended to make it half or a fifth of the unet. If you're training a style you can even set it to 0.
unet_lr = 5e-4 #@param {type:"number"}
text_encoder_lr = 1e-4 #@param {type:"number"}
#@markdown The scheduler is the algorithm that guides the learning rate. If you're not sure, pick `constant` and ignore the number. I personally recommend `cosine_with_restarts` with 3 restarts.
lr_scheduler = "cosine_with_restarts" #@param ["constant", "cosine", "cosine_with_restarts", "constant_with_warmup", "linear", "polynomial"]
lr_scheduler_number = 3 #@param {type:"number"}
lr_scheduler_num_cycles = lr_scheduler_number if lr_scheduler == "cosine_with_restarts" else 0
lr_scheduler_power = lr_scheduler_number if lr_scheduler == "polynomial" else 0
#@markdown Steps spent "warming up" the learning rate during training for efficiency. I recommend leaving it at 5%.
lr_warmup_ratio = 0.05 #@param {type:"slider", min:0.0, max:0.5, step:0.01}
lr_warmup_steps = 0
#@markdown New feature that adjusts loss over time, makes learning much more efficient, and training can be done with about half as many epochs. Uses a value of 5.0 as recommended by [the paper](https://arxiv.org/abs/2303.09556).
min_snr_gamma = True #@param {type:"boolean"}
min_snr_gamma_value = 5.0 if min_snr_gamma else None

#@markdown ### ⚡ Structure
#@markdown LoRA is the classic type and good for a variety of purposes. LoCon is good with artstyles as it has more layers to learn more aspects of the dataset.
lora_type = "LoRA" #@param ["LoRA", "LoCon"]

#@markdown Below are some recommended values for the following settings:

#@markdown | type | network_dim | network_alpha | conv_dim | conv_alpha |
#@markdown | :---: | :---: | :---: | :---: | :---: |
#@markdown | LoRA | 16 | 8 |   |   |
#@markdown | LoCon | 16 | 8 | 8 | 4 |

#@markdown More dim means larger Lora, it can hold more information but more isn't always better. A dim between 8-32 is recommended, and alpha equal to half the dim.
network_dim = 16 #@param {type:"slider", min:1, max:128, step:1}
network_alpha = 8 #@param {type:"slider", min:1, max:128, step:1}
#@markdown The following two values only apply to the additional layers of LoCon.
conv_dim = 8 #@param {type:"slider", min:1, max:64, step:1}
conv_alpha = 4 #@param {type:"slider", min:1, max:64, step:1}

network_module = "networks.lora"
network_args = None
if lora_type.lower() == "locon":
  network_args = [f"conv_dim={conv_dim}", f"conv_alpha={conv_alpha}"]

#@markdown ### ⚡ Ready
#@markdown Ahora que está pronta la configuración podés hacer play para empezar a entrenar tu LoRA, recuerda conectarte a un entorno de ejecución T4 <p>


# 👩‍💻 Cool code goes here

if optimizer.lower() == "prodigy" or "dadapt" in optimizer.lower():
  if override_values_for_dadapt_and_prodigy:
    unet_lr = 0.5
    text_encoder_lr = 0.5
    lr_scheduler = "constant_with_warmup"
    lr_warmup_ratio = 0.05
    network_alpha = network_dim

  if not optimizer_args:
    optimizer_args = ["decouple=True","weight_decay=0.01","betas=[0.9,0.999]"]
    if optimizer == "Prodigy":
      optimizer_args.extend(["d_coef=2","use_bias_correction=True"])
      if lr_warmup_ratio > 0:
        optimizer_args.append("safeguard_warmup=True")
      else:
        optimizer_args.append("safeguard_warmup=False")

root_dir = "/content" if COLAB else "~/Loras"
deps_dir = os.path.join(root_dir, "deps")
repo_dir = os.path.join(root_dir, "kohya-trainer")

if "/Loras" in folder_structure:
  main_dir      = os.path.join(root_dir, "drive/MyDrive/Loras") if COLAB else root_dir
  log_folder    = os.path.join(main_dir, "_logs")
  config_folder = os.path.join(main_dir, project_name)
  images_folder = os.path.join(main_dir, project_name, "dataset")
  output_folder = os.path.join(main_dir, project_name, "output")
else:
  main_dir      = os.path.join(root_dir, "drive/MyDrive/lora_training") if COLAB else root_dir
  images_folder = os.path.join(main_dir, "datasets", project_name)
  output_folder = os.path.join(main_dir, "output", project_name)
  config_folder = os.path.join(main_dir, "config", project_name)
  log_folder    = os.path.join(main_dir, "log")

config_file = os.path.join(config_folder, "training_config.toml")
dataset_config_file = os.path.join(config_folder, "dataset_config.toml")
accelerate_config_file = os.path.join(repo_dir, "accelerate_config/config.yaml")

def install_dependencies():
  os.chdir(root_dir)
  !git clone {SOURCE} {repo_dir}
  os.chdir(repo_dir)
  if COMMIT:
    !git reset --hard {COMMIT}
  !wget https://raw.githubusercontent.com/uYouUs/Hollowstrawberry-kohya-colab/refs/heads/Experiments/train_network_wrapper.py -q -O train_network_wrapper.py
  !wget https://raw.githubusercontent.com/hollowstrawberry/kohya-colab/main/dracula.py -q -O dracula.py

  !apt -y update -qq
  !apt -y install aria2 -qq
  !pip install -r requirements.txt
  if XFORMERS:
    !pip install xformers==0.0.28.post1

  # patch kohya for minor stuff
  if COLAB:
    !sed -i "s@cpu@cuda@" library/model_util.py # low ram
  if LOAD_TRUNCATED_IMAGES:
    !sed -i 's/from PIL import Image/from PIL import Image, ImageFile\nImageFile.LOAD_TRUNCATED_IMAGES=True/g' library/train_util.py # fix truncated jpegs error
  if BETTER_EPOCH_NAMES:
    !sed -i 's/{:06d}/{:02d}/g' library/train_util.py # make epoch names shorter
    !sed -i 's/"." + args.save_model_as)/"-{:02d}.".format(num_train_epochs) + args.save_model_as)/g' train_network.py # name of the last epoch will match the rest

  from accelerate.utils import write_basic_config
  if not os.path.exists(accelerate_config_file):
    write_basic_config(save_location=accelerate_config_file)

  os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
  os.environ["BITSANDBYTES_NOWELCOME"] = "1"
  os.environ["SAFETENSORS_FAST_GPU"] = "1"

def validate_dataset():
  global lr_warmup_steps, lr_warmup_ratio, caption_extension, keep_tokens, keep_tokens_weight, weighted_captions, adjust_tags
  supported_types = (".png", ".jpg", ".jpeg", ".webp", ".bmp")

  print("\n💿 Checking dataset...")
  if not project_name.strip() or any(c in project_name for c in " .()\"'\\/"):
    print("💥 Error: Please choose a valid project name.")
    return

  if custom_dataset:
    try:
      datconf = toml.loads(custom_dataset)
      datasets = [d for d in datconf["datasets"][0]["subsets"]]
    except:
      print(f"💥 Error: Your custom dataset is invalid or contains an error! Please check the original template.")
      return
    reg = [d.get("image_dir") for d in datasets if d.get("is_reg", False)]
    datasets_dict = {d["image_dir"]: d["num_repeats"] for d in datasets}
    folders = datasets_dict.keys()
    files = [f for folder in folders for f in os.listdir(folder)]
    images_repeats = {folder: (len([f for f in os.listdir(folder) if f.lower().endswith(supported_types)]), datasets_dict[folder]) for folder in folders}
  else:
    reg = []
    folders = [images_folder]
    files = os.listdir(images_folder)
    images_repeats = {images_folder: (len([f for f in files if f.lower().endswith(supported_types)]), num_repeats)}

  for folder in folders:
    if not os.path.exists(folder):
      print(f"💥 Error: The folder {folder.replace('/content/drive/', '')} doesn't exist.")
      return
  for folder, (img, rep) in images_repeats.items():
    if not img:
      print(f"💥 Error: Your {folder.replace('/content/drive/', '')} folder is empty.")
      return
  for f in files:
    if not f.lower().endswith((".txt", ".npz")) and not f.lower().endswith(supported_types):
      print(f"💥 Error: Invalid file in dataset: \"{f}\". Aborting.")
      return

  if not [txt for txt in files if txt.lower().endswith(".txt")]:
    caption_extension = ""
  if continue_from_lora and not (continue_from_lora.endswith(".safetensors") and os.path.exists(continue_from_lora)):
    print(f"💥 Error: Invalid path to existing Lora. Example: /content/drive/MyDrive/Loras/example.safetensors")
    return

  pre_steps_per_epoch = sum(img*rep for (img, rep) in images_repeats.values())
  steps_per_epoch = pre_steps_per_epoch/train_batch_size
  total_steps = max_train_steps or int(max_train_epochs*steps_per_epoch)
  estimated_epochs = int(total_steps/steps_per_epoch)
  lr_warmup_steps = int(total_steps*lr_warmup_ratio)

  for folder, (img, rep) in images_repeats.items():
    print("📁"+folder.replace("/content/drive/", "") + (" (Regularization)" if folder in reg else ""))
    print(f"📈 Found {img} images with {rep} repeats, equaling {img*rep} steps.")
  print(f"📉 Divide {pre_steps_per_epoch} steps by {train_batch_size} batch size to get {steps_per_epoch} steps per epoch.")
  if max_train_epochs:
    print(f"🔮 There will be {max_train_epochs} epochs, for around {total_steps} total training steps.")
  else:
    print(f"🔮 There will be {total_steps} steps, divided into {estimated_epochs} epochs and then some.")

  if total_steps > 10000:
    print("💥 Error: Your total steps are too high. You probably made a mistake. Aborting...")
    return

  if adjust_tags:
    print(f"\n📎 Weighted tags: {'ON' if weighted_captions else 'OFF'}")
    if weighted_captions:
      print(f"📎 Will use {keep_tokens_weight} weight on {keep_tokens} activation tag(s)")
    print("📎 Adjusting tags...")
    adjust_weighted_tags(folders, keep_tokens, keep_tokens_weight, weighted_captions)

  return True

def adjust_weighted_tags(folders, keep_tokens: int, keep_tokens_weight: float, weighted_captions: bool):
  weighted_tag = re.compile(r"\((.+?):[.\d]+\)(,|$)")
  for folder in folders:
    for txt in [f for f in os.listdir(folder) if f.lower().endswith(".txt")]:
      with open(os.path.join(folder, txt), 'r') as f:
        content = f.read()
      # reset previous changes
      content = content.replace('\\', '')
      content = weighted_tag.sub(r'\1\2', content)
      if weighted_captions:
        # re-apply changes
        content = content.replace(r'(', r'\(').replace(r')', r'\)').replace(r':', r'\:')
        if keep_tokens_weight > 1:
          tags = [s.strip() for s in content.split(",")]
          for i in range(min(keep_tokens, len(tags))):
            tags[i] = f'({tags[i]}:{keep_tokens_weight})'
          content = ", ".join(tags)
      with open(os.path.join(folder, txt), 'w') as f:
        f.write(content)

def create_config():
  global dataset_config_file, config_file, model_file

  if override_config_file:
    config_file = override_config_file
    print(f"\n⭕ Using custom config file {config_file}")
  else:
    config_dict = {
      "additional_network_arguments": {
        "unet_lr": unet_lr,
        "text_encoder_lr": text_encoder_lr,
        "network_dim": network_dim,
        "network_alpha": network_alpha,
        "network_module": network_module,
        "network_args": network_args,
        "network_train_unet_only": True if text_encoder_lr == 0 else None,
        "network_weights": continue_from_lora if continue_from_lora else None
      },
      "optimizer_arguments": {
        "learning_rate": unet_lr,
        "lr_scheduler": lr_scheduler,
        "lr_scheduler_num_cycles": lr_scheduler_num_cycles if lr_scheduler == "cosine_with_restarts" else None,
        "lr_scheduler_power": lr_scheduler_power if lr_scheduler == "polynomial" else None,
        "lr_warmup_steps": lr_warmup_steps if lr_scheduler != "constant" else None,
        "optimizer_type": optimizer,
        "optimizer_args": optimizer_args if optimizer_args else None,
      },
      "training_arguments": {
        "max_train_steps": max_train_steps,
        "max_train_epochs": max_train_epochs,
        "save_every_n_epochs": save_every_n_epochs,
        "save_last_n_epochs": keep_only_last_n_epochs,
        "train_batch_size": train_batch_size,
        "noise_offset": None,
        "clip_skip": 2,
        "min_snr_gamma": min_snr_gamma_value,
        "weighted_captions": weighted_captions,
        "seed": 42,
        "max_token_length": 225,
        "xformers": XFORMERS,
        "lowram": COLAB,
        "max_data_loader_n_workers": 8,
        "persistent_data_loader_workers": True,
        "save_precision": "fp16",
        "mixed_precision": "fp16",
        "output_dir": output_folder,
        "logging_dir": log_folder,
        "output_name": project_name,
        "log_prefix": project_name,
      },
      "model_arguments": {
        "pretrained_model_name_or_path": model_file,
        "v2": custom_model_is_based_on_sd2,
        "v_parameterization": True if custom_model_is_based_on_sd2 else None,
      },
      "saving_arguments": {
        "save_model_as": "safetensors",
      },
      "dreambooth_arguments": {
        "prior_loss_weight": 1.0,
      },
      "dataset_arguments": {
        "cache_latents": True,
      },
    }

    for key in config_dict:
      if isinstance(config_dict[key], dict):
        config_dict[key] = {k: v for k, v in config_dict[key].items() if v is not None}

    with open(config_file, "w") as f:
      f.write(toml.dumps(config_dict))
    print(f"\n📄 Config saved to {config_file}")

  if override_dataset_config_file:
    dataset_config_file = override_dataset_config_file
    print(f"⭕ Using custom dataset config file {dataset_config_file}")
  else:
    dataset_config_dict = {
      "general": {
        "resolution": resolution,
        "shuffle_caption": shuffle_caption,
        "keep_tokens": keep_tokens,
        "flip_aug": flip_aug,
        "caption_extension": caption_extension,
        "enable_bucket": True,
        "bucket_reso_steps": 64,
        "bucket_no_upscale": False,
        "min_bucket_reso": 320 if resolution > 640 else 256,
        "max_bucket_reso": 1280 if resolution > 640 else 1024,
      },
      "datasets": toml.loads(custom_dataset)["datasets"] if custom_dataset else [
        {
          "subsets": [
            {
              "num_repeats": num_repeats,
              "image_dir": images_folder,
              "class_tokens": None if caption_extension else project_name
            }
          ]
        }
      ]
    }

    for key in dataset_config_dict:
      if isinstance(dataset_config_dict[key], dict):
        dataset_config_dict[key] = {k: v for k, v in dataset_config_dict[key].items() if v is not None}

    with open(dataset_config_file, "w") as f:
      f.write(toml.dumps(dataset_config_dict))
    print(f"📄 Dataset config saved to {dataset_config_file}")

def download_model():
  global old_model_url, model_url, model_file
  real_model_url = model_url.strip()

  if real_model_url.lower().endswith((".ckpt", ".safetensors")):
    model_file = f"/content{real_model_url[real_model_url.rfind('/'):]}"
  else:
    model_file = "/content/downloaded_model.safetensors"
    if os.path.exists(model_file):
      !rm "{model_file}"

  if m := re.search(r"(?:https?://)?(?:www\.)?huggingface\.co/[^/]+/[^/]+/blob", model_url):
    real_model_url = real_model_url.replace("blob", "resolve")
  elif m := re.search(r"(?:https?://)?(?:www\\.)?civitai\.com/models/([0-9]+)(/[A-Za-z0-9-_]+)?", model_url):
    if m.group(2):
      model_file = f"/content{m.group(2)}.safetensors"
    if m := re.search(r"modelVersionId=([0-9]+)", model_url):
      real_model_url = f"https://civitai.com/api/download/models/{m.group(1)}"
    else:
      raise ValueError("optional_custom_training_model_url contains a civitai link, but the link doesn't include a modelVersionId. You can also right click the download button to copy the direct download link.")

  !aria2c "{real_model_url}" --console-log-level=warn -c -s 16 -x 16 -k 10M -d / -o "{model_file}"

  if model_file.lower().endswith(".safetensors"):
    from safetensors.torch import load_file as load_safetensors
    try:
      test = load_safetensors(model_file)
      del test
    except:
      #if "HeaderTooLarge" in str(e):
      new_model_file = os.path.splitext(model_file)[0]+".ckpt"
      !mv "{model_file}" "{new_model_file}"
      model_file = new_model_file
      print(f"Renamed model to {os.path.splitext(model_file)[0]}.ckpt")

  if model_file.lower().endswith(".ckpt"):
    from torch import load as load_ckpt
    try:
      test = load_ckpt(model_file)
      del test
    except:
      return False

  return True

def main():
  global dependencies_installed

  if COLAB and not os.path.exists('/content/drive'):
    from google.colab import drive
    print("📂 Connecting to Google Drive...")
    drive.mount('/content/drive')

  for dir in (main_dir, deps_dir, repo_dir, log_folder, images_folder, output_folder, config_folder):
    os.makedirs(dir, exist_ok=True)

  if not validate_dataset():
    return

  if not dependencies_installed:
    print("\n🏭 Installing dependencies...\n")
    t0 = time()
    install_dependencies()
    t1 = time()
    dependencies_installed = True
    print(f"\n✅ Installation finished in {int(t1-t0)} seconds.")
  else:
    print("\n✅ Dependencies already installed.")

  if old_model_url != model_url or not model_file or not os.path.exists(model_file):
    print("\n🔄 Downloading model...")
    if not download_model():
      print("\n💥 Error: The model you selected is invalid or corrupted, or couldn't be downloaded. You can use a civitai or huggingface link, or any direct download link.")
      return
    print()
  else:
    print("\n🔄 Model already downloaded.\n")

  create_config()

  print("\n⭐ Starting trainer...\n")
  os.chdir(repo_dir)

  !accelerate launch --config_file={accelerate_config_file} --num_cpu_threads_per_process=1 train_network_wrapper.py --dataset_config={dataset_config_file} --config_file={config_file}

main()

Si llegaste hasta acá dentro de tu carpeta mrbeanface deberías tener una carpeta output con una serie de archivos tensorflow, del 1 al 10. Estos son los 10 modelos entrenados de LoRA siendo 10 el más entrenado, que vamos a usar en el paso siguiente

---
🛬 Fin Parte 2 - Desconectarse y eliminar el entorno de ejecución


---



#  🧮 3 - Testeo del LoRA con SD - 🔌 Conectarse a T4

 En este paso generaremos las imágenes sintéticas utilizando un modelo fotorealista basado en SD 1.5 y agregando nuestro LoRA a la cadena de procesamiento de imágen.

##🧮 3.1 - Instalamos las librerías y montamos Google Drive - 🔌Conectarse a GPU


In [ ]:
#Instalar todas las librerías en la Máquina Virtual
!pip install -U peft transformers  # Install the PEFT library
!pip install torch torchvision torchaudio
!pip install diffusers
!pip install transformers
!pip install accelerate
!pip install safetensors
!pip install -U peft transformers

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import torch
from diffusers import DiffusionPipeline
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import textwrap
import os
from datetime import datetime
from matplotlib.font_manager import FontProperties
import torchvision.transforms as T

##🧮 3.2 - Seteamos el nombre del Proyecto

In [ ]:
#@markdown Nombre del proyecto:

project_name = "mrbeanface"  # @param {type:"string"}

project_name = project_name.strip()

##@markdown Esta es la estructura de drive que vamos a usar donde project_name es mrbeanface u otro nombre elegido
folder_structure = "Organize by project (MyDrive/Loras/project_name/dataset)"
## @param ["Organize by category (MyDrive/lora_training/datasets/project_name)", "Organize by project (MyDrive/Loras/project_name/dataset)"]

##🧮 3.3 - Generamos la imagen

In [ ]:


model_1 = "wavymulder/Analog-Diffusion"
model_2= "runwayml/stable-diffusion-v1-5"
model_4 = "dreamlike-art/dreamlike-photoreal-2.0"
model_6 = "SG161222/Realistic_Vision_V2.0"

model_7 = "stablediffusionapi/realistic-vision-v51"
model = model_7


prompt_1 = "photograph of a person at a night party, detailed, 8k"
prompt_2 = "vivid analog Portrait of a person, looking at viewer, joyful mood, serene, dreamy, extremely detailed, stunning portrait, professional photography, early in the morning mist, sun rising behind him, glamorous shot, dynamic pose "
prompt_3 = "young person in a spaceship wearing a space suit with vivid colors 8k"
prompt_7 =  "ultra realistic portrait ((beautiful pale female )), shaved side haircut, hyper detail, cinematic lighting, magic neon, dark red city, in a ia world, futuristic world with robots,  Canon EOS R3, nikon, f/1.4, ISO 200, 1/160s, 8K, RAW, unedited, symmetrical balance, in-frame, 8K"
prompt_8 = "ultra realistic analog  photograph, medium shot, skin moles, 3/4 side, night city, person, smiling, natural skin,  Canon EOS R3, nikon, f/1.4, ISO 200, 1/160s, 8K, RAW"

prompt_9="ultra realistic photo of man, smiling exageratedly, Canon EOS R3, nikon, f/5.6, ISO 200, 1/160s, 8K, RAW"
prompt = prompt_9

negative_prompt ="low quality, bad quality, pixelated, jpeg artifacts, lowres, blurred, poor colors, distorted hands, naked, distorted eyes,lens distortion, close up, painting,digital,selfie, messy hair, cgi,weird eyes,  asian, extra fingers,weird ears, angry, fake skin, shiny skin, mutated hands, poorly drawn hands, poorly drawn face, deformed, ugly, blurry, bad anatomy, bad proportions, extra limbs, cloned face, skinny, glitchy, double torso, extra arms, extra hands, mangled fingers, missing lips, ugly face, latex skin, distorted face, extra legs, anime"

pipe = DiffusionPipeline.from_pretrained(model,torch_dtype=torch.float16).to("cuda")

lora_tensors_path = "/content/drive/MyDrive/Loras/"+project_name+"/output/"+project_name+"-10.safetensors"
lora_weight = 1.2

pipe.load_lora_weights(lora_tensors_path, alpha=lora_weight)
num_inference_steps=300
guidance_scale=2
image = pipe(prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, negative_prompt=negative_prompt).images[0]


display(image)


# Save the image directly to Google Drive with an auto-generated name
save_dir = "/content/drive/MyDrive/Loras/"+project_name+"/"
os.makedirs(save_dir, exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
image_path = os.path.join(save_dir, f"{project_name + timestamp}.png")
image.save(image_path)

del pipe
torch.cuda.empty_cache()


Si llegaste hasta acá seguramente tengas un Mr Bean sintético y sonriente mirandote en la celda de arriba.
Gracias por participar, nos vemos mañana en la charla teórica para compartir críticas y reflexiones de esta nueva tecnología.



---

🚀  Fin del Cuaderno - No olvides desconectar tu entorno de ejecución cuando termines
---

